In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
print(os.getcwd())

/Users/snehitha/taxi_project/notebooks


In [4]:
import sys
import os

# Move up one level and add "src" directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

# Now import the module
import taxi_project.config as config

In [6]:
from taxi_project.inference import get_feature_store

In [7]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-05 08:06:56,676 INFO: Initializing external client
2025-03-05 08:06:56,676 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 08:06:57,550 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215691
Fetching data from 2025-02-04 13:06:56.673702+00:00 to 2025-03-05 12:06:56.673702+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.12s) 


In [8]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 14:00:00+00:00,2,0
1,2025-02-04 15:00:00+00:00,2,0
2,2025-02-04 16:00:00+00:00,2,0
3,2025-02-04 17:00:00+00:00,2,0
4,2025-02-04 18:00:00+00:00,2,0
...,...,...,...
171930,2025-03-04 22:00:00+00:00,263,58
171931,2025-03-04 23:00:00+00:00,263,30
171932,2025-03-05 00:00:00+00:00,263,22
171933,2025-03-05 01:00:00+00:00,263,10


In [9]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171935 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         171935 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  171935 non-null  int32                  
 2   rides               171935 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 3.9 MB


In [10]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [11]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171935 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         171935 non-null  datetime64[us]
 1   pickup_location_id  171935 non-null  int32         
 2   rides               171935 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 3.9 MB


In [12]:
from taxi_project.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,0,0,0,3,0,0,3,5,...,0,0,3,18,0,0,1,0,255,2025-02-14 10:00:00
1,0,0,1,0,0,0,6,2,0,0,...,1,0,2,1,2,0,0,0,112,2025-02-06 07:00:00
2,2,1,0,0,1,1,0,1,0,1,...,0,0,0,1,0,0,0,0,218,2025-02-09 16:00:00
3,369,273,249,21,251,348,17,373,286,251,...,118,9,430,327,65,356,468,212,237,2025-02-19 20:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57,2025-02-05 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,2,0,0,108,2025-03-04 23:00:00
247,0,0,0,2,0,0,4,0,1,0,...,0,2,0,1,0,0,0,0,71,2025-02-23 02:00:00
248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,2025-02-13 13:00:00
249,0,1,1,1,1,1,2,0,4,0,...,0,1,1,3,0,0,0,1,168,2025-02-12 23:00:00


In [14]:
from taxi_project.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-05 08:08:06,523 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 08:08:06,530 INFO: Initializing external client
2025-03-05 08:08:06,530 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 08:08:07,260 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215691
Fetching data from 2025-02-04 13:08:06.523369+00:00 to 2025-03-05 12:08:06.523369+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.78s) 


In [15]:
current_date

Timestamp('2025-03-05 13:08:06.523369+0000', tz='Etc/UTC')

In [16]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 14:00:00
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-04 14:00:00
2,1,7,5,3,2,1,1,5,1,0,...,5,16,11,17,6,1,2,2,4,2025-03-04 14:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 14:00:00
4,2,4,1,0,4,0,0,1,1,2,...,5,7,14,3,3,4,2,1,7,2025-03-04 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,1,0,0,0,0,0,0,0,0,0,...,1,2,0,0,0,0,0,0,259,2025-03-04 14:00:00
247,1,1,0,0,2,0,0,0,0,0,...,1,2,5,1,1,2,1,1,260,2025-03-04 14:00:00
248,37,28,33,48,54,39,24,38,18,9,...,1,9,12,15,31,27,28,27,261,2025-03-04 14:00:00
249,79,97,95,105,104,53,45,21,18,6,...,87,155,104,140,132,103,96,89,262,2025-03-04 14:00:00


In [17]:
from taxi_project.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 08:08:53,148 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 08:08:53,151 INFO: Initializing external client
2025-03-05 08:08:53,151 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 08:08:53,829 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215691


In [18]:
from taxi_project.inference import get_model_predictions
predictions = get_model_predictions(model, features)


In [18]:
from taxi_project.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 02:24:33,567 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 02:24:33,570 INFO: Initializing external client
2025-03-05 02:24:33,570 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 02:24:34,601 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215691
Loading model from /var/folders/1m/z5z5rmbs04j6lnjwzv65l1m80000gn/T/0eebd02e-d67f-4b9f-b92c-b6dae68b34a6/taxi_demand_predictor_next_hour/2/lgb_model.pkl
Loaded model type: <class 'hsml.sklearn.model.Model'>


In [19]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,2.0
3,6,0.0
4,7,1.0
...,...,...
246,259,0.0
247,260,1.0
248,261,29.0
249,262,86.0


In [20]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([237, 161, 236, 132, 138, 162, 170, 163, 142, 140], dtype=int32)

In [21]:
print(predictions.columns)


Index(['pickup_location_id', 'predicted_demand'], dtype='object')


In [22]:
print(predictions.keys())  # Lists all available columns


Index(['pickup_location_id', 'predicted_demand'], dtype='object')
